In [ ]:
!pip install --upgrade cvxpy

In [ ]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt  #cp.multiply랑 array slicing -> etc, [:,0:1]
#initialize variables
delta = 0.5
T = 20
N = int(T/delta) 
K = 4
radius = 5
H =5
pmax = 40
pavg = 10
V = 5
M = 100 # 반복 횟수
w = np.array([[10,0],[0,5],[-10,0],[0,-5]])
e = np.full((1,K),0.06)
#initialize p,q,low
p = cp.Variable((N,1))
q = cp.Variable((N,2))
low = cp.Variable((N,K))  #0~39 는 user 1 , 39~79 : user 2, 79~119 : user 3, 119~159 :user 4
#p의 초기값을 10으로 설정
pin = np.full(N,10.0).reshape(N,1)
p.value = pin
#q의 초기값을 원형을 설정
qin= []
for i in range(0,N,1):
  qin.append([])
  for j in range(2):
    if j==0 :
      qin[i].append(10*np.cos(2*np.pi*i/N))
    elif j==1 :
      qin[i].append(5*np.sin(2*np.pi*i/N))
i=0
qin = np.array(qin).reshape((N,2))
q.value = qin
x = q[:,0].value
y = q[:,1].value
plt.plot(x,y,'ro')
plt.show()
#low값을 초기화  ############################
lowin = np.full(N*K,0.9).reshape(N,K)
low.value = lowin
#ramda & mu setting
b0 =1
eff = 1
r0 = b0/pow(10,-4)
si0 = eff*b0*delta
vramda = np.empty((N,K))
vmu = np.empty((N,K))
def ramda(n,k): #ramda 계산
  vramda[n][k] =  np.sqrt(r0*(1-low[n][k].value)*p[n].value)/(np.power(np.linalg.norm(q[n].value-w[k]),2)+H**2)
  
def mu(n,k) : #mu 계산
  vmu[n][k] = np.sqrt(si0*low[n][k].value*p[n].value)/(np.power(np.linalg.norm(q[n].value-w[k]),2)+H**2)
  #변수들을 초기화시켜주고, 람다와 뮤 값을 계산해주는 함수 생성 완료
pro=0
for m in range(50): # M만큼 반복 시행
  if m>=1 : pro = prob.value
  #stpe 3 : q,p,low를 고정시킨 상태에서, ramda & mu calculate
  for n in range(N):
    for k in range(K):
      ramda(n,k)
      mu(n,k)
  #step 4 : low, ramda, mu를 고정시킨 상태에서 q & p calculate 이때 cvx이용?
  #print("ramda = {}\nmu = {}\n".format(vramda,vmu)) 
  p = cp.Variable((N,1))
  q = cp.Variable((N,2))
  r = cp.Variable()
  obj = cp.Maximize(r)
  #constraint (1) 
  constraints = [ 0<= p, p<= pmax ]
  #constraint (2) 
  constraints += [cp.sum(p) <=pavg*N]
  #constraint (3)
  constraints += [ cp.norm(q[i+1]-q[i],2) <= V*delta for i in range(N-1)]
  #constraint (4) 
  constraints += [q[0] == q[N-1]]
  #constraint (9)
  constraints += [cp.sum(cp.log(1+2*cp.multiply(vramda[:,i:i+1],cp.sqrt(r0*cp.multiply((1-low[:,i:i+1].value),p)))\
                         -cp.multiply(cp.power(vramda[:,i:i+1],2),(cp.power(q[:,0:1]-np.full((N,1),w[i][0]),2)\
                                                                   +cp.power(q[:,1:2]-np.full((N,1),w[i][1]),2)\
                                                   +cp.power(H,2))))/np.log(2))>= r*N for i in range(K)]
  #constraint (10)
  constraints += [ 
                  cp.sum(2*cp.multiply(vmu[:,a:a+1],cp.sqrt(si0*cp.multiply(low[:,a:a+1].value,p)))-cp.multiply(cp.power(vmu[:,a:a+1],2)\
                         ,(cp.power(q[:,0:1]-np.full((N,1),w[a][0]),2)+cp.power(q[:,1:2]\
                                                                                 -np.full((N,1),w[a][1]),2)+cp.power(H,2)))) >= e[0][a]*N for a in range(K)
   ]
  prob = cp.Problem(obj,constraints)
  #print ("step 4 prob is DCP:", prob.is_dcp())
  prob.solve() 

  #print("p.value = {}\nq.value = {}\nlow.value = {}".format(p.value,q.value,low.value))
 #step 5 : q,p,ramda, m를 고정시키고 low, p 구하기 (low를 구한땐 p도 고정?)
  r = cp.Variable()
  low = cp.Variable((N,4))
  #constraint (5)
  constraints = [0<=low,low<=1]
  #constraint (9)
  constraints += [cp.sum(cp.log(1+2*cp.multiply(vramda[:,i:i+1],cp.sqrt(r0*cp.multiply((1-low[:,i:i+1]),p.value)))\
                                -cp.multiply(cp.power(vramda[:,i:i+1],2),\
                                      (cp.power(q[:,0:1].value-np.full((N,1),w[i][0]),2)\
                                       +cp.power(q[:,1:2].value-np.full((N,1),w[i][1]),2)+cp.power(H,2))))/np.log(2))\
                   >= r*N for i in range(K)]
  #constraint (10)
  obj = cp.Maximize(r)
  constraints += [
                  cp.sum(2*cp.multiply(vmu[:,a:a+1],cp.sqrt(si0*cp.multiply(low[:,a:a+1],p.value)))\
                  -cp.multiply(cp.power(vmu[:,a:a+1],2),(cp.power(q[:,0:1].value-np.full((N,1),w[a][0]),2)\
                                                         +cp.power(q[:,1:2].value-np.full((N,1),w[a][1]),2)+cp.power(H,2)))) >=\
                  e[0][a]*N for a in range(K)
   ]
  prob = cp.Problem(obj,constraints)
  #print ("\n\n\nprob5 is DCP:", prob.is_dcp())
  prob.solve()
  print('m= {}'.format(m))
  #print("status:", prob.status)
  print("optimal value", prob.value)
  #print("abs(prob.value-pro) = {}".format(abs(prob.value-pro)))
  #print("p.value = {}\nq.value = {}\nlow.value = {}".format(p.value,q.value,low.value))
  if abs(prob.value-pro)<pow(10,-3): break
  #step 6 : r update

In [ ]:
x = q[:,0].value
y = q[:,1].value
plt.plot(x,y)
plt.axis([-10,10,-5,5])
plt.show()

In [ ]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt  #cp.multiply랑 array slicing -> etc, [:,0:1]
#initialize variables
delta = 0.5
T = 20
N = int(T/delta) 
K = 4
radius = 5
H =5
pmax = 40
pavg = 10
V = 5
M = 100 # 반복 횟수
w = np.array([[10,0],[0,5],[-10,0],[0,-5]])
e = np.full((1,K),0.06)
#initialize p,q,low
p = cp.Variable((N,1))
q = cp.Variable((N,2))
low = cp.Variable((N,K))  #0~39 는 user 1 , 39~79 : user 2, 79~119 : user 3, 119~159 :user 4
#p의 초기값을 10으로 설정
pin = np.full(N,10.0).reshape(N,1)
p.value = pin
#q의 초기값을 원형을 설정
qin= []
for i in range(0,N,1):
  qin.append([])
  for j in range(2):
    if j==0 :
      qin[i].append(10*np.cos(2*np.pi*i/N))
    elif j==1 :
      qin[i].append(5*np.sin(2*np.pi*i/N))
i=0
qin = np.array(qin).reshape((N,2))
q.value = qin
x = q[:,0].value
y = q[:,1].value
plt.plot(x,y,'ro')
plt.show()
#low값을 초기화  ############################
lowin = np.full(N*K,0.9).reshape(N,K)
low.value = lowin
#ramda & mu setting
b0 =1
eff = 1
r0 = b0/pow(10,-4)
si0 = eff*b0*delta
vramda = np.empty((N,K))
vmu = np.empty((N,K))
def ramda(n,k): #ramda 계산
  vramda[n][k] =  np.sqrt(r0*(1-low[n][k].value)*p[n].value)/(np.power(np.linalg.norm(q[n].value-w[k]),2)+H**2)
  
def mu(n,k) : #mu 계산
  vmu[n][k] = np.sqrt(si0*low[n][k].value*p[n].value)/(np.power(np.linalg.norm(q[n].value-w[k]),2)+H**2)
  #변수들을 초기화시켜주고, 람다와 뮤 값을 계산해주는 함수 생성 완료
pro=0
for m in range(5): # M만큼 반복 시행
  if m>=1 : pro = prob.value
  #stpe 3 : q,p,low를 고정시킨 상태에서, ramda & mu calculate
  for n in range(N):
    for k in range(K):
      ramda(n,k)
      mu(n,k)

  q = cp.Variable((N,2))
  r = cp.Variable()
  obj = cp.Maximize(r)
  #constraint (3)
  constraints += [ cp.norm(q[i+1]-q[i],2) <= V*delta for i in range(N-1)]
  #constraint (4) 
  constraints += [q[0] == q[N-1]]
  #constraint (9)
  constraints += [cp.sum(cp.log(1+2*cp.multiply(vramda[:,i:i+1],cp.sqrt(r0*cp.multiply((1-low[:,i:i+1].value),p.value)))\
                         -cp.multiply(cp.power(vramda[:,i:i+1],2),(cp.power(q[:,0:1]-np.full((N,1),w[i][0]),2)\
                                                                   +cp.power(q[:,1:2]-np.full((N,1),w[i][1]),2)\
                                                   +cp.power(H,2))))/np.log(2))>= r*N for i in range(K)]
  #constraint (10)
  constraints += [ 
                  cp.sum(2*cp.multiply(vmu[:,a:a+1],cp.sqrt(si0*cp.multiply(low[:,a:a+1].value,p.value)))-cp.multiply(cp.power(vmu[:,a:a+1],2)\
                         ,(cp.power(q[:,0:1]-np.full((N,1),w[a][0]),2)+cp.power(q[:,1:2]\
                                                                                 -np.full((N,1),w[a][1]),2)+cp.power(H,2)))) >= e[0][a]*N for a in range(K)
   ]
  prob = cp.Problem(obj,constraints)
  print ("step 1 prob is DCP:", prob.is_dcp())
  prob.solve() 

  p = cp.Variable((N,1))
  r = cp.Variable()
  obj = cp.Maximize(r)
  #constraint (1) 
  constraints = [ 0<= p, p<= pmax ]
  #constraint (2) 
  constraints += [cp.sum(p) <=pavg*N]
  #constraint (9)
  constraints += [cp.sum(cp.log(1+2*cp.multiply(vramda[:,i:i+1],cp.sqrt(r0*cp.multiply((1-low[:,i:i+1].value),p)))\
                         -cp.multiply(cp.power(vramda[:,i:i+1],2),(cp.power(q[:,0:1].value-np.full((N,1),w[i][0]),2)\
                                                                   +cp.power(q[:,1:2].value-np.full((N,1),w[i][1]),2)\
                                                   +cp.power(H,2))))/np.log(2))>= r*N for i in range(K)]
  #constraint (10)
  constraints += [ 
                  cp.sum(2*cp.multiply(vmu[:,a:a+1],cp.sqrt(si0*cp.multiply(low[:,a:a+1].value,p)))-cp.multiply(cp.power(vmu[:,a:a+1],2)\
                         ,(cp.power(q[:,0:1].value-np.full((N,1),w[a][0]),2)+cp.power(q[:,1:2].value\
                                                                                 -np.full((N,1),w[a][1]),2)+cp.power(H,2)))) >= e[0][a]*N for a in range(K)
   ]
  prob = cp.Problem(obj,constraints)
  print ("step 2 prob is DCP:", prob.is_dcp())
  prob.solve() 

  r = cp.Variable()
  low = cp.Variable((N,4))
  #constraint (5)
  constraints = [0<=low,low<=1]
  #constraint (9)
  constraints += [cp.sum(cp.log(1+2*cp.multiply(vramda[:,i:i+1],cp.sqrt(r0*cp.multiply((1-low[:,i:i+1]),p.value)))\
                                -cp.multiply(cp.power(vramda[:,i:i+1],2),\
                                      (cp.power(q[:,0:1].value-np.full((N,1),w[i][0]),2)\
                                       +cp.power(q[:,1:2].value-np.full((N,1),w[i][1]),2)+cp.power(H,2))))/np.log(2))\
                   >= r*N for i in range(K)]
  #constraint (10)
  obj = cp.Maximize(r)
  constraints += [
                  cp.sum(2*cp.multiply(vmu[:,a:a+1],cp.sqrt(si0*cp.multiply(low[:,a:a+1],p.value)))\
                  -cp.multiply(cp.power(vmu[:,a:a+1],2),(cp.power(q[:,0:1].value-np.full((N,1),w[a][0]),2)\
                                                         +cp.power(q[:,1:2].value-np.full((N,1),w[a][1]),2)+cp.power(H,2)))) >=\
                  e[0][a]*N for a in range(K)
   ]
  prob = cp.Problem(obj,constraints)
  print ("\n\n\nprob3 is DCP:", prob.is_dcp())
  prob.solve()
  print('m= {}'.format(m))
  #print("status:", prob.status)
  print("optimal value", prob.value)
  #print("abs(prob.value-pro) = {}".format(abs(prob.value-pro)))
  #print("p.value = {}\nq.value = {}\nlow.value = {}".format(p.value,q.value,low.value))
  if abs(prob.value-pro)<pow(10,-3): break
  #step 6 : r update

In [ ]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt  #cp.multiply랑 array slicing -> etc, [:,0:1]
#initialize variables
N = 40 
K = 4
radius = 5
H =5
pmax = 40
pavg = 10
T = 20
delta = T/N
V = 5
M = 100 # 반복 횟수
w = np.array([[10,0],[0,5],[-10,0],[0,-5]])
e = np.full((1,K),0.06)
#initialize p,q,low
p = cp.Variable((N,1))
q = cp.Variable((N,2))
low = cp.Variable((N,K))  #0~39 는 user 1 , 39~79 : user 2, 79~119 : user 3, 119~159 :user 4
#p의 초기값을 10으로 설정
pin = np.full(N,10.0).reshape(N,1)
p.value = pin
#q의 초기값을 원형을 설정
qin= []
for i in range(0,N,1):
  qin.append([])
  for j in range(2):
    if j==0 :
      qin[i].append(10*np.cos(2*np.pi*i/N))
    elif j==1 :
      qin[i].append(5*np.sin(2*np.pi*i/N))
i=0
qin = np.array(qin).reshape((N,2))
q.value = qin
x = q[:,0].value
y = q[:,1].value
plt.plot(x,y,'ro')
plt.show()
#low값을 초기화  ############################
lowin = np.full(N*K,0.9).reshape(N,K)
low.value = lowin
#ramda & mu setting
b0 =1
eff = 1
r0 = b0/pow(10,-4)
si0 = eff*b0*delta
vramda = np.empty((N,K))
vmu = np.empty((N,K))
def ramda(n,k): #ramda 계산
  vramda[n][k] =  np.sqrt(r0*(1-low[n][k].value)*p[n].value)/(np.power(np.linalg.norm(q[n].value-w[k]),2)+H**2)
  
def mu(n,k) : #mu 계산
  vmu[n][k] = np.sqrt(si0*low[n][k].value*p[n].value)/(np.power(np.linalg.norm(q[n].value-w[k]),2)+H**2)
  #변수들을 초기화시켜주고, 람다와 뮤 값을 계산해주는 함수 생성 완료
pro=0
for m in range(3): # M만큼 반복 시행
  if m>=1 : pro = prob.value
  #stpe 3 : q,p,low를 고정시킨 상태에서, ramda & mu calculate
  for n in range(N):
    for k in range(K):
      ramda(n,k)
      mu(n,k)
  #step 4 : low, ramda, mu를 고정시킨 상태에서 q & p calculate 이때 cvx이용?
  print("ramda = {}\nmu = {}\n".format(vramda,vmu)) 
  p = cp.Variable((N,1))
  q = cp.Variable((N,2))
  r = cp.Variable()
  obj = cp.Maximize(r)
  #constraint (1) 
  constraints = [ 0<= p, p<= pmax ]
  #constraint (2) 
  constraints += [cp.sum(p) <=pavg*N]
  #constraint (3)
  constraints += [ cp.norm(q[i+1]-q[i],2) <= V*delta for i in range(N-1)]
  #constraint (4) 
  constraints += [q[0] == q[N-1]]
  #constraint (9)
  constraints += [cp.sum(cp.log(1+2*cp.multiply(vramda[:,i:i+1],cp.sqrt(r0*cp.multiply((1-low[:,i:i+1].value),p)))\
                         -cp.multiply(cp.power(vramda[:,i:i+1],2),(cp.power(q[:,0:1]-np.full((N,1),w[i][0]),2)\
                                                                   +cp.power(q[:,1:2]-np.full((N,1),w[i][1]),2)\
                                                   +cp.power(H,2))))/np.log(2))>= r*N for i in range(K)]
  #constraint (10)
  constraints += [ 
                  cp.sum(2*cp.multiply(vmu[:,a:a+1],cp.sqrt(si0*cp.multiply(low[:,a:a+1].value,p)))-cp.multiply(cp.power(vmu[:,a:a+1],2)\
                         ,(cp.power(q[:,0:1]-np.full((N,1),w[a][0]),2)+cp.power(q[:,1:2]\
                                                                                 -np.full((N,1),w[a][1]),2)+cp.power(H,2)))) >= e[0][a]*N for a in range(K)
   ]
  prob = cp.Problem(obj,constraints)
  print ("step 4 prob is DCP:", prob.is_dcp())
  prob.solve() 

  print("p.value = {}\nq.value = {}\nlow.value = {}".format(p.value,q.value,low.value))
 #step 5 : q,p,ramda, m를 고정시키고 low, p 구하기 (low를 구한땐 p도 고정?)
  r = cp.Variable()
  low = cp.Variable((N,4))
  #constraint (5)
  constraints = [0<=low,low<=1]
  #constraint (9)
  constraints += [cp.sum(cp.log(1+2*cp.multiply(vramda[:,i:i+1],cp.sqrt(r0*cp.multiply((1-low[:,i:i+1]),p.value)))\
                                -cp.multiply(cp.power(vramda[:,i:i+1],2),\
                                      (cp.power(q[:,0:1].value-np.full((N,1),w[i][0]),2)\
                                       +cp.power(q[:,1:2].value-np.full((N,1),w[i][1]),2)+cp.power(H,2))))/np.log(2))\
                   >= r*N for i in range(K)]
  #constraint (10)
  obj = cp.Maximize(r)
  constraints += [
                  cp.sum(2*cp.multiply(vmu[:,a:a+1],cp.sqrt(si0*cp.multiply(low[:,a:a+1],p.value)))\
                  -cp.multiply(cp.power(vmu[:,a:a+1],2),(cp.power(q[:,0:1].value-np.full((N,1),w[a][0]),2)\
                                                         +cp.power(q[:,1:2].value-np.full((N,1),w[a][1]),2)+cp.power(H,2)))) >=\
                  e[0][a]*N for a in range(K)
   ]
  prob = cp.Problem(obj,constraints)
  print ("\n\n\nprob5 is DCP:", prob.is_dcp())
  prob.solve()
  print('m= {}'.format(m))
  #print("status:", prob.status)
  print("optimal value", prob.value)
  #print("abs(prob.value-pro) = {}".format(abs(prob.value-pro)))
  #print("p.value = {}\nq.value = {}\nlow.value = {}".format(p.value,q.value,low.value))
  if abs(prob.value-pro)<pow(10,-3): break
  #step 6 : r update

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(grapx,grapy,color='red')
a = q[:,0].value
b = q[:,1].value
ax2 = ax1.twinx()
ax2.plot(a,b,color='blue')

plt.show()